### Gabriel Ohaike

### Introduction:
 As a data scientist at a game development company, I am interested in tracking two events from my latest mobile game. Buy a sword & join guild. Each has metadata characterstic of such events.

### Tasks:
        
**In order to do this**, 

  1. I will instrument my API server to log events to Kafka
  
  2. Assemble a data pipeline to catch these events using Spark streaming to filter select event types from Kafka, land them into HDFS/parquet to make them available for analysis using Presto
  
  3. Use Apache Bench to generate test data for my pipeline.
 

**Create A docker Compose file**:

The first thing to do is create `docker compose file` that contains all the containers needed to successfully execute the events tracking. The container is made up of **zookeeper, kafka, claudera, spark, presto and mids** container. To see the content structure and port structure please refer to `docker-compose.yml`.

**Here is an example of one of the containers**

In [ ]:
presto:
    image: midsw205/presto:0.0.1
    hostname: presto
    volumes:
      - ~/w205:/w205
    expose:
      - "8080"
    environment:
      HIVE_THRIFTSERVER: cloudera:9083
    extra_hosts:
      - "moby:127.0.0.1"

**Spin up cluster:**

In [ ]:
docker-compose up -d

To spins up the container.The docker-compose up aggregates the output of each container in the docker-compose.yml file and -d starts the containers in the background and leave them running.

**Create a topic:**

In [ ]:
docker-compose exec kafka kafka-topics --create --topic events --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

This code is to create a kafka topic. The exec is use to issue a command expecially when the container is running multiple services. Next, `kafka kafka-topics` tells docker-compose to create a kafka topic. `create --topic events` now create a topc called assessment.partitions 1 allows topics to be parallelized by spitting in data into a particular topic across a multiple brokers. We are only interested in one partition here as per project reqirement, hence the number of `partitions is 1 with the replication factor of 1`. This defines the replication implimented at the partition level. Since we are only interested in one kafka topic, we set our replication factor as 1. --if-not-exists tells the command to execute only if topic does not exist, this avoids errors/warnings.--zookeeper zookeeper:32181 Here the option zookeeper is telling our connection zookeeper to connect to port 32181.

**Create a web-based application:**

In [ ]:
import json
from kafka import KafkaProducer
from flask import Flask, request

app = Flask(__name__)
producer = KafkaProducer(bootstrap_servers='kafka:29092')


def log_to_kafka(topic, event):
    event.update(request.headers)
    producer.send(topic, json.dumps(event).encode())
    
@app.route("/purchase_a_sword")
def purchase_a_sword():
    purchase_sword_event = {'event_type': 'purchase_sword'}
    log_to_kafka('events', purchase_sword_event)
    return "Sword Purchased!\n"

The web app is called `game_api.py` in the folder. The code above shows some of the implementation processes. The `game_api.py` contains three main event. To process this, the `mobile app` makes an `API` call to the `web-based API server` with any of the following calls

   1. `default responses:`
        This returns a default response "This is the default response "
    
   2. `purchase_a_sword:`
        This api is called when the user want to purchase a sword. It ruturns "Sword purchased"
    
   3. `join_a_guild:`
       This is called when a user want to join a guild. It returns "Joined a Guild"
   
 In creating our web-based application, we import `Flask` class and create an instance of class called `app = Flask(__name__)`. We also import `KafkaProducer` to read from kafka using bootstrap.servers configuration to connect to `kafka:29092.` Next, we defined a function `log_to_kafka` to log events to kafka, update event header and use the `send` to send event to kafka `producer` and dump event to `json`before we log it kafka. Encoded with `encode()` for UTF8.  The `route()` decorator tells `Flask` what `url` should trigger the function.

### Streaming Set up

First thing we need to do is getting our set up ready to stream events. To do this, we file

   _1. ab.sh_
   
   _2. guild_sword_stream.py_

### ab.sh:

The `ab.sh` file uses `mids` container with `apache bench` denoted as `ab` to generate data. In the code example below, we are simply using `ab` to generate `150` purchases events from `user1` using a localhost:5000. The `.sh` script controls how we want our events to run during streaming.

In [ ]:
docker-compose exec mids ab -n 150 -H "Host:user1.comcast.com" http://localhost:5000/purchase_a_sword
docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/join_guild

### guild_sword_stream.py

The `guild_sword_stream.py` is used to define events schema, filter out events of interest and load into a `json` file and send to spark to extract events and write it into `HDFS`. Let's look at it in details to understand how different pieces contribute to the overall streaming process.

#### Define events schema:

The code below defines our `schema`. The `StructType` objects is used to define the schema of `Accept,Host, User_Agent, event_type` and flag the nullable for each column in the dataframe to be true. The smooth transition to our `json` file.

In [ ]:
def sword_guild_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
    ])

This extracted events is now fed to streaming mode as request comes i. Here, because our client is only interested in `sword purchases and join guild`, only these two would be filtered. These filtered events is `cast` to string. The `CAST()` function converts a value (of any type) into a specified datatype. In this case, we are converting value from the root to string. Finally,we `write` the stream events to `HDFS` using a processing time of `10 seconds`.

#### UDF 

`@udf` takes boolean values that returns `True or False` . The `purchase_guild` function takes events in json file format and extracts `purchase sword and join guild`. The return `True` returns only `purchase sword and join guild` and filter out any other events.

In [ ]:
@udf('boolean')
def purchase_guild(event_as_json):
    """udf for filtering events
    """
    event = json.loads(event_as_json)
    if event['event_type'] in ('purchase_sword','join_guild'):
        return True
    return False

#### Event Stream

The event streaming is done by using a `SparkSession` to get or create events. These events are then read by `kafka` using a `readStream` to read stream events as they are being fed from `spark`.

In [ ]:
    spark = SparkSession \
        .builder \
        .appName("ExtractEventsJob") \
        .getOrCreate()

    raw_events = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "events") \
        .load()

#### Filtering events

The streaming events from spark is fed to filter `purchase_sword and join_guild` cast them into string and select the filtered events with time stamp from the `json` file as seen in the code below.

In [ ]:
    sword_guild = raw_events \
        .filter(sword_guild(raw_events.value.cast('string'))) \
        .select(raw_events.value.cast('string').alias('raw_event'),
                raw_events.timestamp.cast('string'),
                from_json(raw_events.value.cast('string'),
                          sword_guild_schema()).alias('json')) \
        .select('raw_event', 'timestamp', 'json.*')

#### Sink to hdfs

The events are then written to hdfs using parquet format. The `writestream` writes the stream events after 10 seconds and stores in the `hdfs`. Once all events are written to `hdfs` it terminates using `sink.awaitTermination()`. It is advisable to set the `processing time` to reasonable time in order not to store events in too many small chunk of codes to the `hdfs` or keep so much memory in spark. Checkpoints is saved as `checkpoints_swords_guild`.

In [ ]:
    sink = sword_guild \
        .writeStream \
        .format("parquet") \
        .option("checkpointLocation", "/tmp/checkpoints_swords_guild") \
        .option("path", "/tmp/sword_guild") \
        .trigger(processingTime="10 seconds") \
        .start()

    sink.awaitTermination()